## Deliverable 2. Create a Customer Travel Destinations Map.

In [29]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import os
import numpy as np

# Import API key
from g_config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [30]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv(os.path.join("../Weather_Database", "WeatherPy_Database.csv"))

# Set index to City_ID

city_data_df = city_data_df.set_index("City_ID")

# Preview the DF

city_data_df.head()

,City,Country,Lat,Long,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
City_ID,,,,,,,,,
0,Kapaa,US,22.0752,-159.3190,77.32,78,75,11.50,moderate rain
1,Hovd,MN,48.0056,91.6419,47.10,37,4,7.92,clear sky
2,Hithadhoo,MV,-0.6000,73.0833,81.82,79,72,18.90,light rain
3,Napanee,CA,44.2501,-76.9494,69.84,97,100,8.63,moderate rain
4,Ampanihy,MG,-24.7000,44.7500,68.27,72,37,17.13,scattered clouds


In [32]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 

min_temp = float(input("What is your minimum preferred temperature for your trip? "))
max_temp = float(input("What is your maximum preferred temperature for your trip? "))

What is your minimum preferred temperature for your trip? 75
What is your maximum preferred temperature for your trip? 90


In [33]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.

min_max_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]

# Preview the DataFrame

min_max_df.head(10)

,City,Country,Lat,Long,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
City_ID,,,,,,,,,
0,Kapaa,US,22.0752,-159.3190,77.32,78,75,11.50,moderate rain
2,Hithadhoo,MV,-0.6000,73.0833,81.82,79,72,18.90,light rain
5,Nampula,MZ,-15.1165,39.2666,76.95,57,40,11.50,scattered clouds
6,Ugoofaaru,MV,5.6667,73.0000,83.37,77,99,17.49,overcast clouds
19,Bethel,US,41.3712,-73.4140,79.65,83,75,3.00,broken clouds
22,Sumenep,ID,-7.0167,113.8667,82.09,74,100,12.97,overcast clouds
28,Port Blair,IN,11.6667,92.7500,79.72,81,100,12.21,overcast clouds
29,Vila Velha,BR,-20.3297,-40.2925,75.15,50,75,17.27,broken clouds
31,Victoria,HK,22.2855,114.1577,84.33,87,83,17.74,broken clouds


In [34]:
# 4a. Determine if there are any empty rows.

min_max_df.count()

City                   225
Country                223
Lat                    225
Long                   225
Max Temp               225
Humidity               225
Cloudiness             225
Wind Speed             225
Current Description    225
dtype: int64

In [35]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.

preferred_temp_df = min_max_df.dropna()

# Check the new DF

preferred_temp_df.count()

City                   223
Country                223
Lat                    223
Long                   223
Max Temp               223
Humidity               223
Cloudiness             223
Wind Speed             223
Current Description    223
dtype: int64

In [36]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.

hotel_df = preferred_temp_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Long"]].copy()

# 5b. Create a new column "Hotel Name"

hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Long,Hotel Name
City_ID,,,,,,,
0,Kapaa,US,77.32,moderate rain,22.0752,-159.3190,
2,Hithadhoo,MV,81.82,light rain,-0.6000,73.0833,
5,Nampula,MZ,76.95,scattered clouds,-15.1165,39.2666,
6,Ugoofaaru,MV,83.37,overcast clouds,5.6667,73.0000,
19,Bethel,US,79.65,broken clouds,41.3712,-73.4140,
22,Sumenep,ID,82.09,overcast clouds,-7.0167,113.8667,
28,Port Blair,IN,79.72,overcast clouds,11.6667,92.7500,
29,Vila Velha,BR,75.15,broken clouds,-20.3297,-40.2925,
31,Victoria,HK,84.33,broken clouds,22.2855,114.1577,


In [37]:
# 6a. Set parameters to search for hotels with 5000 meters.

params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.

for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    
    lat = row["Lat"]
    lng = row["Long"]

    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    
    params["location"] = f"{lat},{lng}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    

    # 6e. Make request and retrieve the JSON data from the search. 
    
    hotels = requests.get(base_url, params = params).json()
   
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    
    try:
        
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
        
    except(IndexError, KeyError):
        
        print("Hotel not found... skipping")
        
        hotel_df.loc[index, "Hotel Name"] = np.nan
        

Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping


In [38]:
# Preview the DataFrame

hotel_df

,City,Country,Max Temp,Current Description,Lat,Long,Hotel Name
City_ID,,,,,,,
0,Kapaa,US,77.32,moderate rain,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
2,Hithadhoo,MV,81.82,light rain,-0.6000,73.0833,Scoop Guest House
5,Nampula,MZ,76.95,scattered clouds,-15.1165,39.2666,BUTIK Hotel
6,Ugoofaaru,MV,83.37,overcast clouds,5.6667,73.0000,Ungoofaaru Inn
19,Bethel,US,79.65,broken clouds,41.3712,-73.4140,Courtyard by Marriott Danbury
...,...,...,...,...,...,...,...
664,Rudbar,AF,83.37,clear sky,30.1500,62.6000,NaN
669,Mersing,MY,78.71,broken clouds,2.4312,103.8405,Timotel Hotel
671,Kapatan,PH,78.17,broken clouds,5.9006,125.2375,Hacienda Don Juan


In [39]:
# Count rows to determine NaN values

hotel_df.count()

City                   223
Country                223
Max Temp               223
Current Description    223
Lat                    223
Long                   223
Hotel Name             211
dtype: int64

In [40]:
# 7. Drop the rows where there is no Hotel Name.

hotel_df = hotel_df.dropna()

# Count the rows to test if dropping NaN worked.

hotel_df.count()

City                   211
Country                211
Max Temp               211
Current Description    211
Lat                    211
Long                   211
Hotel Name             211
dtype: int64

In [41]:
# Preview DataFrame Prior to Exporting.

hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Long,Hotel Name
City_ID,,,,,,,
0,Kapaa,US,77.32,moderate rain,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
2,Hithadhoo,MV,81.82,light rain,-0.6000,73.0833,Scoop Guest House
5,Nampula,MZ,76.95,scattered clouds,-15.1165,39.2666,BUTIK Hotel
6,Ugoofaaru,MV,83.37,overcast clouds,5.6667,73.0000,Ungoofaaru Inn
19,Bethel,US,79.65,broken clouds,41.3712,-73.4140,Courtyard by Marriott Danbury
22,Sumenep,ID,82.09,overcast clouds,-7.0167,113.8667,OYO 3271 Homestay Pandian
28,Port Blair,IN,79.72,overcast clouds,11.6667,92.7500,"Welcomhotel By ITC Hotels, Bay Island, Port Blair"
29,Vila Velha,BR,75.15,broken clouds,-20.3297,-40.2925,Quality Suites Vila Velha
31,Victoria,HK,84.33,broken clouds,22.2855,114.1577,Mini Hotel Central


In [43]:
# 8a. Create the output File (CSV)

output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv

hotel_df.to_csv(output_data_file)

In [45]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Long"]]

In [46]:
# 11a. Add a marker layer for each city to the map. 

fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))